#Lego Man vs. Plastic Bag Classifier


In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from google.colab import drive
drive.mount("/content/drive")

import cv2

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls '/content/drive/My Drive/Zümi/Deep Learning Data/tourist'

ls: cannot access '/content/drive/My Drive/Zümi/Deep Learning Data/tourist': No such file or directory


# Setting Initial Parameters

In [0]:
img_width, img_height = 150, 150

train_data_dir = '/content/drive/My Drive/ManvBag/train/'
validation_data_dir = '/content/drive/My Drive/ManvBag/val/'
nb_train_samples = 787 + 786
nb_validation_samples = 236 + 219
epochs = 30
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

#img = cv2.imread('/content/drive/My Drive/train2/1.png',1)
#from IPython.display import Image
#Image('/content/drive/My\ Drive/train2/apple/1.png')

# Neural Network Architecture

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

In [0]:
!ls /content/drive/My\ Drive/ManvBag/train/lego | wc -l

787


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Data Augmentation
Increase the number of training images

In [0]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [0]:

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 1573 images belonging to 2 classes.
Found 455 images belonging to 2 classes.


# Training
"acc" is the accuracy on the training data,
"val_acc" is the accuracy on the testing data

In [0]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
    )

model.save('/content/drive/My Drive/models/manbag2, .h5')

Epoch 1/30
98/98 [==============================] - 454s 5s/step - loss: 0.4338 - acc: 0.7876 - val_loss: 0.2162 - val_acc: 0.9241
Epoch 2/30
98/98 [==============================] - 87s 892ms/step - loss: 0.2238 - acc: 0.9152 - val_loss: 0.0567 - val_acc: 0.9841
Epoch 3/30
98/98 [==============================] - 86s 876ms/step - loss: 0.1326 - acc: 0.9534 - val_loss: 0.0666 - val_acc: 0.9909
Epoch 4/30
98/98 [==============================] - 86s 876ms/step - loss: 0.1203 - acc: 0.9643 - val_loss: 0.0366 - val_acc: 0.9909
Epoch 5/30
98/98 [==============================] - 86s 877ms/step - loss: 0.0864 - acc: 0.9707 - val_loss: 0.0201 - val_acc: 0.9977
Epoch 6/30
98/98 [==============================] - 86s 881ms/step - loss: 0.1527 - acc: 0.9521 - val_loss: 0.0116 - val_acc: 0.9977
Epoch 7/30
98/98 [==============================] - 84s 861ms/step - loss: 0.1313 - acc: 0.9541 - val_loss: 0.0274 - val_acc: 0.9909
Epoch 8/30
98/98 [==============================] - 84s 858ms/step - lo

# Prediction
This prints the Neural Network's best guess of an image

In [0]:
import numpy as np
options={0:"lego man", 1:"plastic bag"}
from keras.preprocessing.image import img_to_array, load_img
img = load_img('/content/drive/My Drive/ManvBag/train/lego/777.jpg',False,target_size=(img_width,img_height))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
preds = model.predict_classes(x)
prob = model.predict_proba(x)
print(options[int(preds)])
print(prob)


lego man
[[1. 0.]]
